In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from plotnine import *
from pylab import rcParams
import matplotlib
from myfunc import timer
from myfunc import DirMan
import config

resolution     = config.resolution
region         = config.region
data_path      = config.data_path
post_data_path = config.post_data_path
shp_path       = config.shp_path
fig_path       = config.fig_path

print('python draw_v1_class.py')

font = {'family': 'Times New Roman'}
matplotlib.rc('font', **font)

params = {'backend': 'ps',
          'axes.labelsize': 25,
          'grid.linewidth': 0.2,
          'font.size': 25,
          'legend.fontsize': 18,
          'legend.frameon': False,
          'xtick.labelsize': 30,
          'xtick.direction': 'out',
          'ytick.labelsize': 30,
          'ytick.direction': 'out',
          'legend.handlelength': 1,
          'legend.handleheight': 1,
          'savefig.bbox': 'tight',
          'axes.unicode_minus': False,
          "mathtext.default":"regular",
          'text.usetex': False}
rcParams.update(params)

df = pd.read_csv(f'{data_path}csv/Global.csv')
shp = gpd.read_file(shp_path+'World_CN/ne_10m_admin_0_countries_chn.shp')

python draw_v1_class.py


In [2]:
df2 = df.copy()

df2['Continent_Together'] = df2['Continent'].replace(to_replace=['South America','North America'], value=['South \nAmerica','North \nAmerica']) 

df3 = pd.DataFrame()
df3['Continent_area'] = df2.groupby('Continent_Together')['Area'].sum().div(1e9)
# df3 = df3[df3['Continent_Together'] > 300]
df3 = df3.sort_values(by=['Continent_area'], ascending=False).reset_index(drop=False)
print(df3)

list1 = df3.loc[:,'Continent_Together']
list2 = df3.loc[:,'Continent_area']
mapping = dict(zip(list1, list2))

df2 = df2[df2.Continent_Together.isin(list1)]
df2 = df2[df2['Continent_Together'].notna()]
df2['Continent_area'] = df2['Continent_Together'].map(mapping)
df2 = df2.sort_values(by=['Continent_area'], ascending=False).reset_index(drop=True)
order = df2['Continent_Together'].unique()
df2['Continent_Together'] = pd.Categorical(df2['Continent_Together'], categories=order, ordered=True)
print(df2)
    
violin_plot = (ggplot(df2, aes(x='Continent_Together', y="Sbedrock", fill="Continent_Together"))
        + geom_violin(show_legend=False)
        + geom_boxplot(fill="white", width=0.1, show_legend=False, outlier_alpha=0, outlier_size=1, outlier_color='#f6f2f4')
        + scale_fill_hue(s=0.90, l=0.65, h=0.0417, color_space='husl')
        #    + coord_flip()
        + theme_matplotlib()
        + ggtitle("Continent")
        + theme(  # legend_position='none',
text=element_text(size=20, colour="black"),
axis_text_x=element_text(angle=90, hjust=0.5),
axis_title_x=element_text(vjust=1),
plot_title=element_text(vjust=1, hjust = 0.5),
aspect_ratio=0.8,
dpi=400,
figure_size=(16, 8))
        + labs(x='', y="$S_{{bedrock}}$ (mm)"))
# violin_plot.save(f'{fig_path}v1_con.pdf', transparent=True, bbox_inches='tight')
violin_plot.save(f'{fig_path}v1_con.png')

  Continent_Together  Continent_area
0               Asia        0.001124
1             Africa        0.000424
2    North \nAmerica        0.000418
3    South \nAmerica        0.000331
4             Europe        0.000293
5          Australia        0.000133
6            Oceania        0.000010
        Unnamed: 0    lat     lon    Sbedrock          Sr       Ssoil  \
0           169814  72.75  104.65    7.494956   92.492600   84.997110   
1           169852  75.35   96.35    8.161146   42.961930   34.798187   
2           169851  74.75   94.95   56.937702   56.941536    0.000000   
3           169850  74.65   92.55   15.885224   44.881016   28.991125   
4           169849  74.45  108.95   59.713875   59.717600    0.000000   
...            ...    ...     ...         ...         ...         ...   
169815       32963  -5.55  148.85  170.810730  252.789930   81.977800   
169816       33320  -5.35  149.95  118.593710  227.981800  109.384920   
169817       33713  -5.15  150.05  454.573850  

/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 16 x 8 in image.
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: /home/xuxh22/stu01/Bedrock/fig/0p1/v1_con.png


In [3]:
df2 = df.copy()

list1 = ['South America','Australia and New Zealand','Southern Africa','Eastern Africa' ,'Melanesia', 
        'Western Europe', 'Polynesia', 'Middle Africa','South-Eastern Asia', 'Western Africa', 
        'Southern Asia', 'Central America','Northern Africa', 'Caribbean', 'Western Asia', 
        'Eastern Asia','Northern America', 'Southern Europe', 'Central Asia', 'Eastern Europe',
        'Northern Europe']
list2 = ['South \nAmerica','Australia and \nNew Zealand','Southern \nAfrica','Eastern \nAfrica' ,'Melanesia', 
        'Western \nEurope', 'Polynesia', 'Middle \nAfrica','South-Eastern \nAsia', 'Western \nAfrica', 
        'Southern \nAsia', 'Central \nAmerica','Northern \nAfrica', 'Caribbean', 'Western \nAsia', 
        'Eastern \nAsia','Northern \nAmerica', 'Southern \nEurope', 'Central \nAsia', 'Eastern \nEurope',
        'Northern \nEurope']
mapping = dict(zip(list1, list2))
df2['Subregion_Together'] = df2['Subregion'].map(mapping).fillna(df2['Subregion'])

df3 = pd.DataFrame()
df3['Subregion_area'] = df2.groupby('Subregion_Together')['Area'].sum().div(1e9)
df3 = df3[df3['Subregion_area'] > 300]
df3 = df3.sort_values(by=['Subregion_area'], ascending=False).reset_index(drop=False)
print(df3)

list1 = df3.loc[:,'Subregion_Together']
list2 = df3.loc[:,'Subregion_area']
mapping = dict(zip(list1, list2))

df2 = df2[df2.Subregion_Together.isin(list1)]
df2 = df2[df2['Subregion_Together'].notna()]
df2['Subregion_area'] = df2['Subregion_Together'].map(mapping)
df2 = df2.sort_values(by=['Subregion_area'], ascending=False).reset_index(drop=True)
order = df2['Subregion_Together'].unique()
df2['Subregion_Together'] = pd.Categorical(df2['Subregion_Together'], categories=order, ordered=True)
print(df2)
    
violin_plot = (ggplot(df2, aes(x='Subregion_Together', y="Sbedrock", fill="Subregion_Together"))
        + geom_violin(show_legend=False)
        + geom_boxplot(fill="white", width=0.1, show_legend=False, outlier_alpha=0, outlier_size=1, outlier_color='#f6f2f4')
        + scale_fill_hue(s=0.90, l=0.65, h=0.0417, color_space='husl')
        #    + coord_flip()
        + theme_matplotlib()
        + ggtitle("Subregion")
        
        + theme(  # legend_position='none',
text=element_text(size=20, colour="black"),
axis_text_x=element_text(angle=90, hjust=0.5),
axis_title_x=element_text(vjust=1),
plot_title=element_text(vjust=1, hjust = 0.5),
aspect_ratio=0.8,
dpi=400,
figure_size=(16, 8))
        + labs(x='', y="$S_{{bedrock}}$ (mm)"))
# violin_plot.save(f'{fig_path}v1_sub.pdf', transparent=True, bbox_inches='tight')
violin_plot.save(f'{fig_path}v1_sub.png')

Empty DataFrame
Columns: [Subregion_Together, Subregion_area]
Index: []
Empty DataFrame
Columns: [Unnamed: 0, lat, lon, Sbedrock, Sr, Ssoil, Area, Continent, Subregion, Sovereignt, Continent_short, Subregion_short, Sovereignt_short, Subregion_Together, Subregion_area]
Index: []


/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 16 x 8 in image.
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: /home/xuxh22/stu01/Bedrock/fig/0p1/v1_sub.png


TypeError: object of type 'NoneType' has no len()